<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/notebooks/label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install transformers

In [2]:
# ! pip install datasets

In [3]:
from transformers import pipeline

In [4]:
classifier = pipeline("zero-shot-classification", device = 0)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
candidate_labels = ['sports', 'others']

In [13]:
from tqdm import tqdm

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [22]:
from transformers.pipelines.pt_utils import KeyDataset

In [36]:
dataset_subset = dataset['test'].shard(num_shards=10, index=0)

In [44]:
dataset_subset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1149
})

In [49]:
{'label': labels[0]['labels'][0], 'score': labels[0]['scores'][0]}

{'label': 'others', 'score': 0.6010456085205078}

In [ ]:
for out in tqdm(classifier(KeyDataset(dataset['test'], "article"), candidate_labels)):
    labels.append({'label': out['labels'][0], 'score': out['scores'][0]})

  1%|          | 99/11490 [00:42<1:35:32,  1.99it/s]

In [45]:
labels

[{'sequence': 'f001ec5c4704938247d27a44948eebb37ae98d01',
  'labels': ['others', 'entertainment'],
  'scores': [0.6010456085205078, 0.3989543616771698]},
 {'sequence': 'ee17dfb574feca82ccac5689595e47483bd23f12',
  'labels': ['others', 'entertainment'],
  'scores': [0.6544768214225769, 0.3455232083797455]},
 {'sequence': 'ffff11a2f44d731cd80c86819a89b7e227581415',
  'labels': ['others', 'entertainment'],
  'scores': [0.6842106580734253, 0.3157893121242523]},
 {'sequence': '63e19d477469e39e3fc4143919e6a5acb58dbc23',
  'labels': ['others', 'entertainment'],
  'scores': [0.7124776840209961, 0.2875223159790039]},
 {'sequence': 'f7c2eb465479b965669b81535234e24735915ebf',
  'labels': ['others', 'entertainment'],
  'scores': [0.6192398071289062, 0.38076019287109375]},
 {'sequence': '8012ab9261e117ab47a2759b3c0b628f2a007a82',
  'labels': ['others', 'entertainment'],
  'scores': [0.6958884000778198, 0.3041115999221802]},
 {'sequence': '599bf6f13bc8477d50aa7599baf53123a6e57708',
  'labels': ['oth